In [1]:
#Import the libraries
from textblob import TextBlob
import nltk
import pandas as pd
from textblob import TextBlob
import pandas as pd
import string
import re
from string import digits
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
import gensim
import matplotlib.pyplot as plt
from stemming.porter2 import stem

ModuleNotFoundError: No module named 'stemming'

In [12]:
import io 
import codecs
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
file1 = codecs.open('hong_kong_protest_reddit.csv','r','utf-8') 
line = file1.read() 
words = line.split()
for r in words: 
    if not r in stop_words: 
        appendFile = open('hong_protest_reddit.csv','a', encoding='utf-8') 
        appendFile.write(r)
        appendFile.write("\n")
        appendFile.close()

In [13]:
HongKong = pd.read_csv( 'hong_kong_protest_reddit.csv')
HongKong = HongKong.dropna()

In [14]:
HongKong.head()

,comments
0,Users often report submissions from this site ...
1,>“The authorities should stipulate what issues...
2,"He's a well known Chinese nationalist, this sh..."
3,"This is the best tl;dr I could make, [original..."
4,>When you see the name Jackie Chan you know yo...


In [15]:
def preprocessing(x):
    x = x.lower()
#     x = x.replace('—', ' ')
    x = x.replace('-', ' ')
    x = re.sub('\w+@\w+\.[a-z]{3}', ' ', x)
    x = x.translate(str.maketrans(" ", " ", string.punctuation))
    x = x.translate(str.maketrans('', '', digits))
    x = x.replace('“', '')
    x = x.replace('”', '')
    x = x.replace('…', '')
    x = x.replace('’', '')
    x = x.replace('‘', '')  
    x = ' '.join(x.split())
    x = x.split('—')
#     print(x)
    if(x[0].isupper()):
        x = x[1:]
    x = ' '.join(x)
    x = x.strip()
    x = x.replace('  ', '')
    return x
HongKong['comments'].apply(preprocessing)

0        users often report submissions from this site ...
1        the authorities should stipulate what issues p...
2        hes a well known chinese nationalist this shou...
3        this is the best tldr i could make originalhtt...
4        when you see the name jackie chan you know you...
5        china is a totalitarian regime jackie chan is ...
6        hes very involved in the ccp and has always be...
7                         lost a lot of respect for jackie
8        he made a movie called looking for jackie that...
9        once you find out who he really is and what ch...
10                                                 deleted
11       as i understood he has long been a chinese gov...
12       fuck that guy this is not the first time he ha...
13                   but what does ja think about all this
14       worlds most concussed man has terrible opinion...
15              chinesw man gives chinese opinions more at
16       lets also not forget that jackie is a media ce.

In [19]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

HongKong['comments'].apply(get_stemmed_text)
HongKong.head()

,comments
0,Users often report submissions from this site ...
1,>“The authorities should stipulate what issues...
2,"He's a well known Chinese nationalist, this sh..."
3,"This is the best tl;dr I could make, [original..."
4,>When you see the name Jackie Chan you know yo...


In [22]:
comments = HongKong['comments']
counter = 0
positive_sentiment_array = []
negative_sentiment_array = []
sentiment_score_array = []
label_array = []
topic_array = []
for comment in comments:
    counter += 1
    obj = TextBlob( comment )
    sentiment = obj.sentiment.polarity
    if sentiment > 0.0:
            positive_sentiment_array.append(obj)
            label_array.append("positive")
    if sentiment <0.0:
            negative_sentiment_array.append(obj)
            label_array.append("negative")
    if sentiment == 0.0:
            label_array.append('neutral')
    sentiment_score_array.append(sentiment)
    topic_array.append("Hong Kong Protest")
    
#returns the sentiment of text
#by returning a value between -1.0 and 1.0

print(len(positive_sentiment_array))
print(len(negative_sentiment_array))
print(counter)

34826
20199
78094


In [23]:
HongKong.insert(1, "Score", sentiment_score_array , True) 
HongKong.insert(2, "Label", label_array , True) 
HongKong.insert(3, "Topic", topic_array , True) 

In [24]:
HongKong.head()

,comments,Score,Label,Topic
0,Users often report submissions from this site ...,-0.075000,negative,Hong Kong Protest
1,>“The authorities should stipulate what issues...,0.800000,positive,Hong Kong Protest
2,"He's a well known Chinese nationalist, this sh...",0.000000,neutral,Hong Kong Protest
3,"This is the best tl;dr I could make, [original...",0.322222,positive,Hong Kong Protest
4,>When you see the name Jackie Chan you know yo...,0.114286,positive,Hong Kong Protest


In [26]:
HongKong.to_csv('HongKongProtestSentimentAnalysis.csv',encoding='utf-8-sig')